# Getting started with Word2Vec in Gensim and making it work!

The idea behind Word2Vec is pretty simple. We are making and assumption that you can tell the meaning of a word by the company it keeps. This is analogous to the saying *show me your friends, and I'll tell who you are*. So if you have two words that have very similar neighbors (i.e. the usage context is about the same), then these words are probably quite similar in meaning or are at least highly related. For example, the words `shocked`,`appalled` and `astonished` are typically used in a similar context. 

In this tutorial, you will learn how to use the Gensim implementation of Word2Vec and actually get it to work! I have heard a lot of complaints about poor performance etc, but its really a combination of two things, (1) your input data and (2) your parameter settings. Note that the training algorithms in this package were ported from the [original Word2Vec implementation by Google](https://arxiv.org/pdf/1301.3781.pdf) and extended with additional functionality.

### Imports and logging

First, we start with our imports and get logging established:

In [1]:
# imports needed and set up logging
import gzip
import gensim 
import logging

logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s', level=logging.INFO)


### Dataset 
Next, is our dataset. The secret to getting Word2Vec really working for you is to have lots and lots of text data. In this case I am going to use data from the [OpinRank](http://kavita-ganesan.com/entity-ranking-data/) dataset. This dataset has full user reviews of cars and hotels. I have specifically concatenated all of the hotel reviews into one big file which is about 97MB compressed and 229MB uncompressed. We will use the compressed file for this tutorial. Each line in this file represents a hotel review. You can download the OpinRank Word2Vec dataset here.

To avoid confusion, while gensim’s word2vec tutorial says that you need to pass it a sequence of sentences as its input, you can always pass it a whole review as a sentence (i.e. a much larger size of text), and it should not make much of a difference. 

Now, let's take a closer look at this data below by printing the first line. You can see that this is a pretty hefty review.

In [2]:
data_file="reviews_data.txt.gz"

with gzip.open ('reviews_data.txt.gz', 'rb') as f:
    for i,line in enumerate (f):
        print(line)
        break


b"Oct 12 2009 \tNice trendy hotel location not too bad.\tI stayed in this hotel for one night. As this is a fairly new place some of the taxi drivers did not know where it was and/or did not want to drive there. Once I have eventually arrived at the hotel, I was very pleasantly surprised with the decor of the lobby/ground floor area. It was very stylish and modern. I found the reception's staff geeting me with 'Aloha' a bit out of place, but I guess they are briefed to say that to keep up the coroporate image.As I have a Starwood Preferred Guest member, I was given a small gift upon-check in. It was only a couple of fridge magnets in a gift box, but nevertheless a nice gesture.My room was nice and roomy, there are tea and coffee facilities in each room and you get two complimentary bottles of water plus some toiletries by 'bliss'.The location is not great. It is at the last metro stop and you then need to take a taxi, but if you are not planning on going to see the historic sites in Be

### Read files into a list
Now that we've had a sneak peak of our dataset, we can read it into a list so that we can pass this on to the Word2Vec model. Notice in the code below, that I am directly reading the 
compressed file. I'm also doing a mild pre-processing of the reviews using `gensim.utils.simple_preprocess (line)`. This does some basic pre-processing such as tokenization, lowercasing, etc and returns back a list of tokens (words). Documentation of this pre-processing method can be found on the official [Gensim documentation site](https://radimrehurek.com/gensim/utils.html). 



In [13]:

def read_input(input_file):
    """This method reads the input file which is in gzip format"""
    
    logging.info("reading file {0}...this may take a while".format(input_file))
    
    with gzip.open (input_file, 'rb') as f:
        for i, line in enumerate (f): 

            if (i%10000==0):
                logging.info ("read {0} reviews".format (i))
            # do some pre-processing and return a list of words for each review text
            yield gensim.utils.simple_preprocess (line)

# read the tokenized reviews into a list
# each review item becomes a serries of words
# so this becomes a list of lists
documents = list (read_input (data_file))
logging.info ("Done reading data file")  
print(len(documents))

2021-01-08 10:57:02,741 : INFO : reading file reviews_data.txt.gz...this may take a while
2021-01-08 10:57:02,743 : INFO : read 0 reviews
2021-01-08 10:57:09,739 : INFO : read 10000 reviews
2021-01-08 10:57:18,468 : INFO : read 20000 reviews
2021-01-08 10:57:27,889 : INFO : read 30000 reviews
2021-01-08 10:57:36,114 : INFO : read 40000 reviews
2021-01-08 10:57:45,516 : INFO : read 50000 reviews
2021-01-08 10:57:54,396 : INFO : read 60000 reviews
2021-01-08 10:58:02,132 : INFO : read 70000 reviews
2021-01-08 10:58:08,762 : INFO : read 80000 reviews
2021-01-08 10:58:16,313 : INFO : read 90000 reviews
2021-01-08 10:58:23,135 : INFO : read 100000 reviews
2021-01-08 10:58:30,292 : INFO : read 110000 reviews
2021-01-08 10:58:37,933 : INFO : read 120000 reviews
2021-01-08 10:58:44,944 : INFO : read 130000 reviews
2021-01-08 10:58:52,796 : INFO : read 140000 reviews
2021-01-08 10:58:59,669 : INFO : read 150000 reviews
2021-01-08 10:59:07,098 : INFO : read 160000 reviews
2021-01-08 10:59:15,491

<class 'list'>
255404


## Training the Word2Vec model

Training the model is fairly straightforward. You just instantiate Word2Vec and pass the reviews that we read in the previous step (the `documents`). So, we are essentially passing on a list of lists. Where each list within the main list contains a set of tokens from a user review. Word2Vec uses all these tokens to internally create a vocabulary. And by vocabulary, I mean a set of unique words.

After building the vocabulary, we just need to call `train(...)` to start training the Word2Vec model. Training on the [OpinRank](http://kavita-ganesan.com/entity-ranking-data/) dataset takes about 10 minutes so please be patient while running your code on this dataset.

Behind the scenes we are actually training a simple neural network with a single hidden layer. But, we are actually not going to use the neural network after training. Instead, the goal is to learn the weights of the hidden layer. These weights are essentially the word vectors that we’re trying to learn. 

In [15]:
model = gensim.models.Word2Vec (documents, size=150, window=10, min_count=2, workers=10)
model.train(documents,total_examples=len(documents),epochs=10)

2021-01-08 11:42:27,261 : WARNING : consider setting layer size to a multiple of 4 for greater performance
2021-01-08 11:42:27,263 : INFO : collecting all words and their counts
2021-01-08 11:42:27,264 : INFO : PROGRESS: at sentence #0, processed 0 words, keeping 0 word types
2021-01-08 11:42:27,628 : INFO : PROGRESS: at sentence #10000, processed 1655714 words, keeping 25777 word types
2021-01-08 11:42:27,980 : INFO : PROGRESS: at sentence #20000, processed 3317863 words, keeping 35016 word types
2021-01-08 11:42:28,420 : INFO : PROGRESS: at sentence #30000, processed 5264072 words, keeping 47518 word types
2021-01-08 11:42:28,802 : INFO : PROGRESS: at sentence #40000, processed 7081746 words, keeping 56675 word types
2021-01-08 11:42:29,268 : INFO : PROGRESS: at sentence #50000, processed 9089491 words, keeping 63744 word types
2021-01-08 11:42:29,673 : INFO : PROGRESS: at sentence #60000, processed 11013726 words, keeping 76786 word types
2021-01-08 11:42:30,042 : INFO : PROGRESS: a

2021-01-08 11:43:55,282 : INFO : EPOCH 1 - PROGRESS: at 63.20% examples, 511731 words/s, in_qsize 15, out_qsize 4
2021-01-08 11:43:56,315 : INFO : EPOCH 1 - PROGRESS: at 65.24% examples, 512019 words/s, in_qsize 18, out_qsize 2
2021-01-08 11:43:57,326 : INFO : EPOCH 1 - PROGRESS: at 66.92% examples, 512258 words/s, in_qsize 18, out_qsize 1
2021-01-08 11:43:58,339 : INFO : EPOCH 1 - PROGRESS: at 68.70% examples, 512246 words/s, in_qsize 19, out_qsize 0
2021-01-08 11:43:59,436 : INFO : EPOCH 1 - PROGRESS: at 70.63% examples, 513288 words/s, in_qsize 10, out_qsize 9
2021-01-08 11:44:00,435 : INFO : EPOCH 1 - PROGRESS: at 72.62% examples, 515082 words/s, in_qsize 18, out_qsize 2
2021-01-08 11:44:01,442 : INFO : EPOCH 1 - PROGRESS: at 74.44% examples, 515126 words/s, in_qsize 19, out_qsize 0
2021-01-08 11:44:02,454 : INFO : EPOCH 1 - PROGRESS: at 76.03% examples, 514729 words/s, in_qsize 14, out_qsize 1
2021-01-08 11:44:03,459 : INFO : EPOCH 1 - PROGRESS: at 77.60% examples, 514418 words/s,

2021-01-08 11:45:00,182 : INFO : EPOCH 2 - PROGRESS: at 73.51% examples, 509548 words/s, in_qsize 18, out_qsize 1
2021-01-08 11:45:01,208 : INFO : EPOCH 2 - PROGRESS: at 75.15% examples, 508941 words/s, in_qsize 16, out_qsize 3
2021-01-08 11:45:02,213 : INFO : EPOCH 2 - PROGRESS: at 76.77% examples, 509295 words/s, in_qsize 18, out_qsize 1
2021-01-08 11:45:03,212 : INFO : EPOCH 2 - PROGRESS: at 78.52% examples, 510218 words/s, in_qsize 16, out_qsize 3
2021-01-08 11:45:04,227 : INFO : EPOCH 2 - PROGRESS: at 80.46% examples, 511904 words/s, in_qsize 19, out_qsize 0
2021-01-08 11:45:05,251 : INFO : EPOCH 2 - PROGRESS: at 82.24% examples, 512073 words/s, in_qsize 20, out_qsize 0
2021-01-08 11:45:06,268 : INFO : EPOCH 2 - PROGRESS: at 83.88% examples, 511613 words/s, in_qsize 16, out_qsize 3
2021-01-08 11:45:07,266 : INFO : EPOCH 2 - PROGRESS: at 85.57% examples, 512046 words/s, in_qsize 18, out_qsize 1
2021-01-08 11:45:08,275 : INFO : EPOCH 2 - PROGRESS: at 87.37% examples, 511869 words/s,

2021-01-08 11:46:05,458 : INFO : EPOCH 3 - PROGRESS: at 84.47% examples, 516813 words/s, in_qsize 19, out_qsize 0
2021-01-08 11:46:06,486 : INFO : EPOCH 3 - PROGRESS: at 86.16% examples, 516527 words/s, in_qsize 18, out_qsize 1
2021-01-08 11:46:07,501 : INFO : EPOCH 3 - PROGRESS: at 88.47% examples, 518666 words/s, in_qsize 19, out_qsize 0
2021-01-08 11:46:08,538 : INFO : EPOCH 3 - PROGRESS: at 90.39% examples, 518846 words/s, in_qsize 19, out_qsize 0
2021-01-08 11:46:09,577 : INFO : EPOCH 3 - PROGRESS: at 92.29% examples, 518686 words/s, in_qsize 16, out_qsize 3
2021-01-08 11:46:10,596 : INFO : EPOCH 3 - PROGRESS: at 93.95% examples, 518359 words/s, in_qsize 17, out_qsize 2
2021-01-08 11:46:11,598 : INFO : EPOCH 3 - PROGRESS: at 95.78% examples, 518454 words/s, in_qsize 17, out_qsize 2
2021-01-08 11:46:12,636 : INFO : EPOCH 3 - PROGRESS: at 97.55% examples, 518267 words/s, in_qsize 19, out_qsize 2
2021-01-08 11:46:13,660 : INFO : EPOCH 3 - PROGRESS: at 99.39% examples, 518235 words/s,

2021-01-08 11:47:10,429 : INFO : EPOCH 4 - PROGRESS: at 95.74% examples, 515633 words/s, in_qsize 17, out_qsize 2
2021-01-08 11:47:11,472 : INFO : EPOCH 4 - PROGRESS: at 97.83% examples, 516942 words/s, in_qsize 13, out_qsize 6
2021-01-08 11:47:12,413 : INFO : worker thread finished; awaiting finish of 9 more threads
2021-01-08 11:47:12,463 : INFO : worker thread finished; awaiting finish of 8 more threads
2021-01-08 11:47:12,485 : INFO : EPOCH 4 - PROGRESS: at 99.82% examples, 517874 words/s, in_qsize 3, out_qsize 9
2021-01-08 11:47:12,492 : INFO : worker thread finished; awaiting finish of 7 more threads
2021-01-08 11:47:12,494 : INFO : worker thread finished; awaiting finish of 6 more threads
2021-01-08 11:47:12,496 : INFO : worker thread finished; awaiting finish of 5 more threads
2021-01-08 11:47:12,498 : INFO : worker thread finished; awaiting finish of 4 more threads
2021-01-08 11:47:12,502 : INFO : worker thread finished; awaiting finish of 3 more threads
2021-01-08 11:47:12,50

...

2021-01-08 11:52:15,894 : INFO : EPOCH 6 - PROGRESS: at 19.23% examples, 797737 words/s, in_qsize 20, out_qsize 1
2021-01-08 11:52:16,910 : INFO : EPOCH 6 - PROGRESS: at 21.61% examples, 797537 words/s, in_qsize 18, out_qsize 1
2021-01-08 11:52:17,906 : INFO : EPOCH 6 - PROGRESS: at 23.76% examples, 801344 words/s, in_qsize 18, out_qsize 1
2021-01-08 11:52:18,905 : INFO : EPOCH 6 - PROGRESS: at 26.57% examples, 804334 words/s, in_qsize 19, out_qsize 0
2021-01-08 11:52:19,908 : INFO : EPOCH 6 - PROGRESS: at 29.55% examples, 806169 words/s, in_qsize 18, out_qsize 1
2021-01-08 11:52:20,921 : INFO : EPOCH 6 - PROGRESS: at 32.59% examples, 807791 words/s, in_qsize 18, out_qsize 1
2021-01-08 11:52:21,921 : INFO : EPOCH 6 - PROGRESS: at 35.48% examples, 810462 words/s, in_qsize 19, out_qsize 0
2021-01-08 11:52:22,936 : INFO : EPOCH 6 - PROGRESS: at 38.46% examples, 812282 words/s, in_qsize 19, out_qsize 0
2021-01-08 11:52:23,980 : INFO : EPOCH 6 - PROGRESS: at 41.40% examples, 810272 words/s,

2021-01-08 11:53:20,417 : INFO : EPOCH 7 - PROGRESS: at 90.18% examples, 774742 words/s, in_qsize 15, out_qsize 4
2021-01-08 11:53:21,446 : INFO : EPOCH 7 - PROGRESS: at 92.60% examples, 772056 words/s, in_qsize 20, out_qsize 2
2021-01-08 11:53:22,478 : INFO : EPOCH 7 - PROGRESS: at 94.97% examples, 769509 words/s, in_qsize 19, out_qsize 0
2021-01-08 11:53:23,483 : INFO : EPOCH 7 - PROGRESS: at 97.32% examples, 767004 words/s, in_qsize 19, out_qsize 0
2021-01-08 11:53:24,455 : INFO : worker thread finished; awaiting finish of 9 more threads
2021-01-08 11:53:24,486 : INFO : EPOCH 7 - PROGRESS: at 99.81% examples, 765574 words/s, in_qsize 5, out_qsize 7
2021-01-08 11:53:24,520 : INFO : worker thread finished; awaiting finish of 8 more threads
2021-01-08 11:53:24,523 : INFO : worker thread finished; awaiting finish of 7 more threads
2021-01-08 11:53:24,527 : INFO : worker thread finished; awaiting finish of 6 more threads
2021-01-08 11:53:24,530 : INFO : worker thread finished; awaiting f

2021-01-08 11:54:16,428 : INFO : EPOCH 9 - PROGRESS: at 33.39% examples, 765820 words/s, in_qsize 18, out_qsize 1
2021-01-08 11:54:17,446 : INFO : EPOCH 9 - PROGRESS: at 36.40% examples, 771746 words/s, in_qsize 19, out_qsize 0
2021-01-08 11:54:18,443 : INFO : EPOCH 9 - PROGRESS: at 39.36% examples, 775901 words/s, in_qsize 19, out_qsize 0
2021-01-08 11:54:19,474 : INFO : EPOCH 9 - PROGRESS: at 42.40% examples, 778250 words/s, in_qsize 17, out_qsize 2
2021-01-08 11:54:20,491 : INFO : EPOCH 9 - PROGRESS: at 45.50% examples, 781245 words/s, in_qsize 18, out_qsize 1
2021-01-08 11:54:21,536 : INFO : EPOCH 9 - PROGRESS: at 48.45% examples, 783635 words/s, in_qsize 18, out_qsize 1
2021-01-08 11:54:22,527 : INFO : EPOCH 9 - PROGRESS: at 51.32% examples, 785730 words/s, in_qsize 19, out_qsize 2
2021-01-08 11:54:23,553 : INFO : EPOCH 9 - PROGRESS: at 53.71% examples, 783172 words/s, in_qsize 20, out_qsize 1
2021-01-08 11:54:24,570 : INFO : EPOCH 9 - PROGRESS: at 56.37% examples, 780749 words/s,

2021-01-08 11:55:18,514 : INFO : worker thread finished; awaiting finish of 8 more threads
2021-01-08 11:55:18,519 : INFO : worker thread finished; awaiting finish of 7 more threads
2021-01-08 11:55:18,523 : INFO : worker thread finished; awaiting finish of 6 more threads
2021-01-08 11:55:18,525 : INFO : worker thread finished; awaiting finish of 5 more threads
2021-01-08 11:55:18,526 : INFO : worker thread finished; awaiting finish of 4 more threads
2021-01-08 11:55:18,526 : INFO : worker thread finished; awaiting finish of 3 more threads
2021-01-08 11:55:18,527 : INFO : worker thread finished; awaiting finish of 2 more threads
2021-01-08 11:55:18,528 : INFO : worker thread finished; awaiting finish of 1 more threads
2021-01-08 11:55:18,531 : INFO : worker thread finished; awaiting finish of 0 more threads
2021-01-08 11:55:18,532 : INFO : EPOCH - 10 : training on 41519358 raw words (30350701 effective words) took 37.6s, 806811 effective words/s
2021-01-08 11:55:18,533 : INFO : trainin

(303494842, 415193580)

## Now, let's look at some output 
This first example shows a simple case of looking up words similar to the word `dirty`. All we need to do here is to call the `most_similar` function and provide the word `dirty` as the positive example. This returns the top 10 similar words. 

In [40]:

w1 = ["sweet", "filthy"]
try:
    res = model.wv.most_similar (positive=w1[1], topn=10)
    print(res)
except KeyError as e:
    print(e)


[('dirty', 0.8704475164413452), ('disgusting', 0.8161634802818298), ('stained', 0.7779815196990967), ('smelly', 0.6976768970489502), ('gross', 0.6882807016372681), ('disgustingly', 0.6876232624053955), ('discusting', 0.6857088804244995), ('soiled', 0.6856330633163452), ('unclean', 0.6770219802856445), ('mouldy', 0.6692030429840088)]


That looks pretty good, right? Let's look at a few more. Let's look at similarity for `polite`, `france` and `shocked`. 

In [44]:
# look up top 6 words similar to 'polite'
w1 = "polite"
model.wv.most_similar (positive=w1,topn=6)


[('courteous', 0.9256340861320496),
 ('friendly', 0.8343440890312195),
 ('cordial', 0.805959939956665),
 ('professional', 0.7815728187561035),
 ('attentive', 0.775558352470398),
 ('curteous', 0.7616664171218872)]

In [53]:
# look up top 6 words similar to 'france'
w1 = ["france"]
model.wv.most_similar (positive=w1,topn=6)


[('canada', 0.6603403091430664),
 ('germany', 0.6510637998580933),
 ('spain', 0.6431018114089966),
 ('barcelona', 0.61174076795578),
 ('mexico', 0.6070996522903442),
 ('rome', 0.6065913438796997)]

In [54]:
# look up top 6 words similar to 'shocked'
w1 = ["shocked"]
model.wv.most_similar (positive=w1,topn=6)


[('horrified', 0.80775386095047),
 ('amazed', 0.7797470092773438),
 ('astonished', 0.7748459577560425),
 ('dismayed', 0.7680633068084717),
 ('stunned', 0.7603034973144531),
 ('appalled', 0.7466776371002197)]

That's, nice. You can even specify several positive examples to get things that are related in the provided context and provide negative examples to say what should not be considered as related. In the example below we are asking for all items that *relate to bed* only:

In [61]:
# get everything related to stuff on the bed
w1 = ["bed",'sheet','pillow']
w2 = ['blanket']
model.wv.most_similar (positive=w1, negative=w2, topn=10)


[('mattress', 0.6979163885116577),
 ('matress', 0.6603406667709351),
 ('cushion', 0.6419447660446167),
 ('duvet', 0.6333613395690918),
 ('pillows', 0.602047324180603),
 ('beds', 0.6007523536682129),
 ('satin', 0.5868145823478699),
 ('quilt', 0.5840531587600708),
 ('cushions', 0.5682719945907593),
 ('bolster', 0.5605882406234741)]

### Similarity between two words in the vocabulary

You can even use the Word2Vec model to return the similarity between two words that are present in the vocabulary. 

In [57]:
# similarity between two different words
model.wv.similarity(w1="dirty",w2="smelly")

0.76181122646029453

In [58]:
# similarity between two identical words
model.wv.similarity(w1="dirty",w2="dirty")

1.0000000000000002

In [59]:
# similarity between two unrelated words
model.wv.similarity(w1="dirty",w2="clean")

0.25355593501920781

Under the hood, the above three snippets computes the cosine similarity between the two specified words using word vectors of each. From the scores, it makes sense that `dirty` is highly similar to `smelly` but `dirty` is dissimilar to `clean`. If you do a similarity between two identical words, the score will be 1.0 as the range of the cosine similarity score will always be between [0.0-1.0]. You can read more about cosine similarity scoring [here](https://en.wikipedia.org/wiki/Cosine_similarity).

### Find the odd one out
You can even use Word2Vec to find odd items given a list of items.

In [63]:
# Which one is the odd one out in this list?
model.wv.doesnt_match(["cat","dog","france"])

'france'

In [77]:
# Which one is the odd one out in this list?
model.wv.doesnt_match(["bed","pillow","duvet","shower"])


'shower'

## Understanding some of the parameters
To train the model earlier, we had to set some parameters. Now, let's try to understand what some of them mean. For reference, this is the command that we used to train the model.

```
model = gensim.models.Word2Vec (documents, size=150, window=10, min_count=2, workers=10)
```

### `size`
The size of the dense vector to represent each token or word. If you have very limited data, then size should be a much smaller value. If you have lots of data, its good to experiment with various sizes. A value of 100-150 has worked well for me. 

### `window`
The maximum distance between the target word and its neighboring word. If your neighbor's position is greater than the maximum window width to the left and the right, then, some neighbors are not considered as being related to the target word. In theory, a smaller window should give you terms that are more related. If you have lots of data, then the window size should not matter too much, as long as its a decent sized window. 

### `min_count`
Minimium frequency count of words. The model would ignore words that do not statisfy the `min_count`. Extremely infrequent words are usually unimportant, so its best to get rid of those. Unless your dataset is really tiny, this does not really affect the model.

### `workers`
How many threads to use behind the scenes?


## When should you use Word2Vec?

There are many application scenarios for Word2Vec. Imagine if you need to build a sentiment lexicon. Training a Word2Vec model on large amounts of user reviews helps you achieve that. You have a lexicon for not just sentiment, but for most words in the vocabulary. 

Beyond, raw unstructured text data, you could also use Word2Vec for more structured data. For example, if you had tags for a million stackoverflow questions and answers, you could find tags that are related to a given tag and recommend the related ones for exploration. You can do this by treating each set of co-occuring tags as a "sentence" and train a Word2Vec model on this data. Granted, you still need a large number of examples to make it work. 
